In [2]:
!pip3 install --user 'implicit'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 1.2 MB/s 


In [ ]:
%matplotlib inline
import os
import sys
import re
import random
import matplotlib
import implicit

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from time import time

os.environ["OPENBLAS_NUM_THREADS"] = "1"   

base = "/mnt/datasets/lastfm/"

In [ ]:
userArtistDF = pd.read_csv(base + 'user_artist_data.txt', 
                           sep=' ', 
                           header=None, 
                           names=['userID', 'artistID', 'playCount'])

userArtistDF['userID'] = userArtistDF['userID'].astype(np.int64, errors='ignore')
userArtistDF['artistID'] = userArtistDF['artistID'].astype(np.int64, errors='ignore')
userArtistDF['playCount'] = userArtistDF['playCount'].astype(np.int32, errors='ignore')

userArtistDF[:5]

In [ ]:
uniqueUsers = userArtistDF['userID'].nunique()
print("Total n. of users: ", uniqueUsers)

In [ ]:
uniqueArtists = userArtistDF['artistID'].nunique()
print("Total n. of users: ", uniqueArtists)

In [ ]:
userActivity= userArtistDF.groupby('userID')['playCount'].sum()
print(userActivity[0:5])

In [ ]:
XX = df['playCount']
YY = np.arange(len(XX))/float(len(XX))


per = [10, 25, 50, 75, 90]
colors = ['royalblue','mediumblue','darkblue','navy','black']
rank = [int(x / 100 * (len(YY) + 1)) for x in per]
rplot = [YY[x] for x in rank]
pervalues_users = [XX[x] for x in rank]

plt.figure(figsize=(12, 6))


ax = plt.subplot()
ax.set_xscale("log")
plt.plot(XX, YY)

 
plt.scatter(pervalues_users,
            rplot, 
            c = colors,
            marker = 'o')
ax.vlines(pervalues_users, [0], rplot, color=colors, linestyle='--')
ax.hlines(rplot, [0], pervalues_users, color=colors, linestyle='--')
for i in range(len(rplot)):
    ax.annotate("  " + str(pervalues_users[i]), (pervalues_users[i], rplot[i]))
    
plt.xlabel('Play Counts in log')
plt.ylabel('ECDF')
plt.grid(True,which="both",ls="-")
plt.title('ECDF of number of play counts per User ID')
plt.show()


plt.figure(figsize=(12, 10))
plt.boxplot(XX)
plt.yscale('log')
plt.ylabel('Play Counts in log')
plt.title('The statistical distribution')
plt.show()

In [ ]:
artistPopularity = userArtistDF.groupby('artistID')['playCount'].sum()


df = artistPopularity.to_frame().sort_values(by=['playCount']).reset_index() 
print(df.quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])) 

XX = df['playCount']
YY = np.arange(len(XX))/float(len(XX))

per = [10, 25, 50, 75, 90]
colors = ['royalblue','mediumblue','darkblue','navy','black']
rank = [int(x / 100 * (len(YY) + 1)) for x in per]
rplot = [YY[x] for x in rank]
pervalues_users = [XX[x] for x in rank]

plt.figure(figsize=(12, 6))

ax = plt.subplot()
ax.set_xscale("log")
plt.plot(XX, YY)


plt.scatter(pervalues_users,
            rplot, 
            c = colors,
            marker = 'o')
ax.vlines(pervalues_users, [0], rplot, color=colors, linestyle='--')
ax.hlines(rplot, [0], pervalues_users, color=colors, linestyle='--')
for i in range(len(rplot)):
    ax.annotate("  " + str(pervalues_users[i]), (pervalues_users[i], rplot[i]))
    
plt.xlabel('Play Counts in log')
plt.ylabel('ECDF')
plt.grid(True,which="both",ls="-")
plt.title('ECDF of number of play counts per Artist ID')
plt.show()


plt.figure(figsize=(12, 10))
plt.boxplot(XX)
plt.yscale('log')
plt.ylabel('Play Counts in log')
plt.title('The statistical distribution')
plt.show()

In [ ]:
topN = 10
topNArtist = df[-topN:]
print(topNArtist)


plt.barh(range(len(topNArtist)),topNArtist['playCount'])
plt.yticks(range(len(topNArtist)), topNArtist['artistID'])

plt.xlabel('Play Counts')
plt.ylabel('Artist')
plt.title('Top-10 Artist ID per play counts')
plt.grid(True,which="both",ls="-")

plt.show()

In [ ]:
artistDF = pd.read_csv(base + 'artist_data.txt', 
                           sep='\t', 
                           header=None, 
                           names=['artistID', 'name'])

artistDF['artistID'] = artistDF['artistID'].astype(np.int64, errors='ignore')
artistDF['name'] = artistDF['name'].astype(np.str, errors='ignore')

artistDF = artistDF.dropna()
pd.to_numeric(artistDF['name'],errors='ignore')

artistDF[:5]

In [ ]:
artistDF[artistDF['name'].str.contains('Aerosmith')][:20]

In [ ]:
artistDF[artistDF['artistID'] == '1000010']

In [ ]:
artistDF[artistDF['artistID'] == '2082323']

In [ ]:
artistAliasDF = pd.read_csv(base + 'artist_alias.txt', 
                           sep='\t', 
                           header=None, 
                           names=['misspelledArtistID', 'standardArtistID'])

artistAliasDF['misspelledArtistID'] = artistAliasDF['misspelledArtistID'].astype(np.int64, errors='ignore')
artistAliasDF['standardArtistID'] = artistAliasDF['standardArtistID'].astype(np.int64, errors='ignore')


artistAliasDF[:5]

In [ ]:
artistAliasDF[artistAliasDF['misspelledArtistID'] == 1000010]

In [ ]:
artistAliasDF[artistAliasDF['misspelledArtistID'] == 2082323]

In [ ]:
t0 = time()

newUserArtistDF = pd.merge(userArtistDF, artistAliasDF, left_on='artistID', right_on='misspelledArtistID', how='left')

newUserArtistDF.loc[newUserArtistDF['misspelledArtistID'].isnull(),'standardArtistID'] = newUserArtistDF['artistID'] 

newUserArtistDF['realArtistID'] = newUserArtistDF['standardArtistID'].astype(np.int64)

newUserArtistDF.drop(['artistID', 'standardArtistID', 'misspelledArtistID'], axis=1, inplace=True)

newUserArtistDF[:5]

In [ ]:
len(newUserArtistDF)

In [ ]:
newUserArtistDF[:5]

In [ ]:
len(newUserArtistDF)

In [ ]:
assert (newUserArtistDF.groupby(['userID', 'realArtistID'], as_index=False).count()['playCount'] == 1).all(),  \
    'Each combination user/artist should appear only once'

In [ ]:
uniqueArtists = newUserArtistDF['realArtistID'].nunique()
print("Total n. of artists: ", uniqueArtists)

In [ ]:
topN = 10


artistPopularity = newUserArtistDF.groupby('realArtistID')['playCount'].sum() 
artistPopularity = artistPopularity.to_frame().reset_index()  
artistPopularity['realArtistID'] = artistPopularity['realArtistID'].astype(str)

artistPopularity = pd.merge(artistPopularity, artistDF, right_on='artistID', left_on='realArtistID', how='left')
topNArtist = artistPopularity.sort_values(by=['playCount'], ascending=False)[:topN]


plt.barh(range(len(topNArtist)),topNArtist['playCount'])
plt.yticks(range(len(topNArtist)), topNArtist['name'])

plt.xlabel('Play Counts')
plt.ylabel('Artist')
plt.title('Top-10 Artist per play counts')
plt.grid(True,which="both",ls="-")

plt.show()